# High Resolution inference 

In [1]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

/private/home/hadyelsahar/work/code/videoseal/videoseal


In [2]:
from videoseal.utils.display import save_vid
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.data.datasets import VideoDataset
import os
from tqdm import tqdm
import torch
import gc

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu" 

/private/home/hadyelsahar/anaconda3/envs/videoseal/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from tqdm import tqdm
import torch

# Directory containing videos
video_dir = "assets/videos/metamoviegen"

# Example usage
ckpts = {
    "hidden": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.0_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=hidden/checkpoint.pth',
    "unet": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth'
}

fps = 24 // 1
frames_per_clip = fps * 3  # 3s
frame_step = 1

# Iterate over all checkpoints
for model_name, ckpt in ckpts.items():
    wam = setup_model_from_checkpoint(ckpt)
    wam.eval()
    wam.to(device)

    # Iterate over all video files in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")][1:]

    for video_file in tqdm(video_files, desc=f"Processing Videos for {model_name}"):
        video_path = os.path.join(video_dir, video_file)
        base_name = os.path.splitext(video_file)[0]

        # Load video (assuming a function `load_video` exists)
        vid, mask = VideoDataset.load_full_video_decord(video_path)

        outputs = wam.embed(vid, is_video=True)

        # Move outputs to CPU
        imgs = vid.cpu()  # b c h w
        imgs_w = outputs["imgs_w"].cpu()  # b c h w

        diff = imgs_w - imgs

        save_vid(imgs, f"{model_name}_{base_name}.mp4", fps)
        save_vid(imgs_w, f"{model_name}_{base_name}_wmed.mp4", fps)
        save_vid(diff, f"{model_name}_{base_name}_wm.mp4", fps)

        # Compute min and max values, reshape, and normalize
        min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        normalized_images = (diff - min_vals) / (max_vals - min_vals)

        # Save the normalized video
        save_vid(normalized_images, f"{model_name}_{base_name}_wm_normalized.mp4", fps)

        # Free GPU memory
        del outputs
        del imgs_w
        del diff
        del normalized_images
        torch.cuda.empty_cache()

        del vid
        del mask
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()

    # Free model from GPU
    del wam
    torch.cuda.empty_cache()

Model loaded successfully from /private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.0_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=hidden/checkpoint.pth


Processing Videos for hidden: 100%|██████████| 3/3 [02:44<00:00, 54.81s/it]


Model loaded successfully from /private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth


Processing Videos for unet: 100%|██████████| 3/3 [03:00<00:00, 60.04s/it]
